In [49]:
# process the input data to fit yolo training image format
import json
import numpy as np 
import os

DATA_PATH = 'dataset/all_data'
OUTPUT_PATH = 'dataset/yolo_hand_data'

In [30]:
# 
from PIL import Image, ImageDraw

with Image.open(pic_jpg) as img:
  draw = ImageDraw.Draw(img)

  hand_pts = data['hand_pts']
  left_pt = min(hand_pts, key=lambda hand_pt: hand_pt[0])
  right_pt = max(hand_pts, key=lambda hand_pt: hand_pt[0])
  up_pt = min(hand_pts, key=lambda hand_pt: hand_pt[1])
  low_pt = max(hand_pts, key=lambda hand_pt: hand_pt[1])
  upper_left_coordinate = (left_pt[0], up_pt[1])
  lower_right_coordinate = (right_pt[0], low_pt[1])
  draw.rectangle((upper_left_coordinate, lower_right_coordinate), outline=(255, 255, 255))

  # tuple_hand_pts = tuple((hand_pt[0], hand_pt[1]) for hand_pt in hand_pts)

  # draw.point(tuple((hand_pt[0], hand_pt[1]) for hand_pt in hand_pts), fill=(255, 255, 0))
  # draw.point(((upmost_pt[0], upmost_pt[1])), fill=(255, 255, 0))


  img.show()

In [50]:
import glob
from sklearn.model_selection import train_test_split

seed = 1

jpg_file_paths = glob.glob(f'{DATA_PATH}/*.jpg')
jpg_file_names = []

for jpg_file_path in jpg_file_paths:
  jpg_file_names.append(os.path.split(jpg_file_path)[1])

file_names = list(map(lambda jpg_filename: jpg_filename.strip('.jpg'), jpg_file_names))
json_file_names = list(map(lambda filename: f'{filename}.json', file_names))
# partition the data into training, validation and test set
filenames_train, filenames_valid, label_train, img_valid = train_test_split(file_names, json_file_names, test_size=0.1, random_state=seed)

# img_train, img_valid, label_train, label_valid = train_test_split(img_train, label_train, test_size=1/9, random_state=seed)

In [51]:
filenames_train[4]

'Ricki_unit_8.flv_000126_l'

In [52]:
print(f'training set length: {len(img_train)}')
print(f'validation(test) set length: {len(img_valid)}')
# print(f'test set length: {len(img_test)}')

training set length: 2206
validation(test) set length: 276


In [63]:
import shutil
from tqdm import tqdm

def create_yolo_standard_dataset(filenames, data_type='train'):
  # for each filename
  # extract the hand info from the json file create a txt for it
  # copy the file from all_data/ into yolo_hand_data/images/{data_type}
  for filename in tqdm(filenames):
    json_file_path = f'{DATA_PATH}/{filename}.json'
    jpg_file_path = f'{DATA_PATH}/{filename}.jpg' 
    
    with open(json_file_path) as json_file:
      data = json.load(json_file)
      yolo_data = []

      # get the hand points in the image files
      hand_pts = data['hand_pts']
      left_pt = min(hand_pts, key=lambda hand_pt: hand_pt[0])
      right_pt = max(hand_pts, key=lambda hand_pt: hand_pt[0])
      up_pt = min(hand_pts, key=lambda hand_pt: hand_pt[1])
      low_pt = max(hand_pts, key=lambda hand_pt: hand_pt[1])
      upper_left_coordinate = (left_pt[0], up_pt[1])
      lower_right_coordinate = (right_pt[0], low_pt[1])
      
      h = lower_right_coordinate[1] - upper_left_coordinate[1]
      w = lower_right_coordinate[0] - upper_left_coordinate[0]

      x_center = left_pt[0] + w / 2
      y_center = up_pt[1] + h / 2

      with Image.open(jpg_file_path) as jpg_file:
        width, height = jpg_file.size
      x_center /= width
      y_center /= height

      yolo_data.append([0, x_center, y_center, w, h])

      yolo_data = np.array(yolo_data)
      np.savetxt(
        os.path.join(OUTPUT_PATH, f'labels/{data_type}/{filename}.txt'),
        yolo_data, 
        fmt=['%d', '%f', '%f', '%f', '%f'])
      shutil.copy(jpg_file_path, os.path.join(OUTPUT_PATH, f'images/{data_type}/{filename}.jpg'))


In [64]:
create_yolo_standard_dataset(filenames_train, data_type='train')
create_yolo_standard_dataset(filenames_valid, data_type='validation')

100%|██████████| 276/276 [00:10<00:00, 26.44it/s]


In [7]:
list(map(lambda jpg_filename: jpg_filename.split('.')[0], jpg_filenames))

['dataset/all_data/012162289_01_l',
 'dataset/all_data/056067829_01_l',
 'dataset/all_data/064868295_01_l',
 'dataset/all_data/Jackie_unit_10',
 'dataset/all_data/085013273_02_r',
 'dataset/all_data/092535491_01_l',
 'dataset/all_data/072399538_02_r',
 'dataset/all_data/039447939_02_l',
 'dataset/all_data/ex2_2',
 'dataset/all_data/041008968_01_l',
 'dataset/all_data/Alexander_a_visit_to_the_cafe',
 'dataset/all_data/Alexander_a_visit_to_the_cafe',
 'dataset/all_data/089422804_01_r',
 'dataset/all_data/003913225_02_l',
 'dataset/all_data/Berry_roof_story',
 'dataset/all_data/013566147_01_r',
 'dataset/all_data/052680292_02_r',
 'dataset/all_data/064544183_02_l',
 'dataset/all_data/048386022_02_l',
 'dataset/all_data/Alexander_mouse_cat_rooster',
 'dataset/all_data/088283649_01_r',
 'dataset/all_data/036124279_02_r',
 'dataset/all_data/020446672_02_l',
 'dataset/all_data/018259713_01_r',
 'dataset/all_data/010280856_02_r',
 'dataset/all_data/002076409_01_l',
 'dataset/all_data/Berry_ser